# MPI for Sonar simulation

## Imports

In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')

In [2]:
%%px --no-stream --group-outputs=engine

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(f"Hi, I'm rank {rank}")

[stdout:0] 
Hi, I'm rank 0
[stdout:1] 
Hi, I'm rank 1


In [3]:
%%px --no-stream --group-outputs=engine

import sys
import os
import matplotlib.pyplot as plt
import numpy as np

sys.path.insert(0, os.path.abspath('/home/hajta2/study/sonar-FWI/cli/'))
from simulation.sonar import Sonar
from simulation.utils import EllipsisBottom, gen_velocity_profile
from simulation.sources import GaborSource

In [4]:
%%px --no-stream --group-outputs=engine
from devito import configuration
configuration['mpi'] = True
configuration['language'] = 'C'
# configuration['platform'] = 'nvidiaX'
# configuration['log-level'] = 'DEBUG'
# os.environ['DEVITO_ARCH'] = 'nvc'

## Initialization

In [5]:
%%px --no-stream --group-outputs=engine

domain_size = (60, 30)
v_env = 1.5
ns = 128
source_distance = 0.002
f0 = 5
space_order = 8
spatial_dist = round(v_env / f0 / 3, 3)
dt = spatial_dist / 20

In [6]:
%%px --no-stream --group-outputs=engine

sonar = Sonar(domain_size, f0, v_env, EllipsisBottom(True), source_distance=source_distance, ns=ns, spatial_dist=spatial_dist)

[stderr:0] 
51it [00:00, 1390.49it/s]
Operator `initdamp` ran in 0.01 s
[stderr:1] 
51it [00:00, 1351.84it/s]
Operator `initdamp` ran in 0.01 s


In [7]:
%%px --no-stream --group-outputs=engine

sonar.set_source()
sonar.finalize()

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

In [8]:
%%px

sonar.src.coordinates.data

Out[0:7]: 
Data([[29.872     ,  0.129     ],
      [29.87401575,  0.129     ],
      [29.8760315 ,  0.129     ],
      [29.87804724,  0.129     ],
      [29.88006299,  0.129     ],
      [29.88207874,  0.129     ],
      [29.88409449,  0.129     ],
      [29.88611024,  0.129     ],
      [29.88812598,  0.129     ],
      [29.89014173,  0.129     ],
      [29.89215748,  0.129     ],
      [29.89417323,  0.129     ],
      [29.89618898,  0.129     ],
      [29.89820472,  0.129     ],
      [29.90022047,  0.129     ],
      [29.90223622,  0.129     ],
      [29.90425197,  0.129     ],
      [29.90626772,  0.129     ],
      [29.90828346,  0.129     ],
      [29.91029921,  0.129     ],
      [29.91231496,  0.129     ],
      [29.91433071,  0.129     ],
      [29.91634646,  0.129     ],
      [29.9183622 ,  0.129     ],
      [29.92037795,  0.129     ],
      [29.9223937 ,  0.129     ],
      [29.92440945,  0.129     ],
      [29.9264252 ,  0.129     ],
      [29.92844094,  0.129     ],
   

Out[1:7]: 
Data([[30.00100787,  0.129     ],
      [30.00302362,  0.129     ],
      [30.00503937,  0.129     ],
      [30.00705512,  0.129     ],
      [30.00907087,  0.129     ],
      [30.01108661,  0.129     ],
      [30.01310236,  0.129     ],
      [30.01511811,  0.129     ],
      [30.01713386,  0.129     ],
      [30.01914961,  0.129     ],
      [30.02116535,  0.129     ],
      [30.0231811 ,  0.129     ],
      [30.02519685,  0.129     ],
      [30.0272126 ,  0.129     ],
      [30.02922835,  0.129     ],
      [30.03124409,  0.129     ],
      [30.03325984,  0.129     ],
      [30.03527559,  0.129     ],
      [30.03729134,  0.129     ],
      [30.03930709,  0.129     ],
      [30.04132283,  0.129     ],
      [30.04333858,  0.129     ],
      [30.04535433,  0.129     ],
      [30.04737008,  0.129     ],
      [30.04938583,  0.129     ],
      [30.05140157,  0.129     ],
      [30.05341732,  0.129     ],
      [30.05543307,  0.129     ],
      [30.05744882,  0.129     ],
   

In [9]:
%%px

src = sonar.src
alpha = 45
dt = sonar.model.critical_dt
ns = src.coordinates.data.shape[0]

if alpha <= 90:
    max_latency = (
        np.cos(np.deg2rad(alpha)) * ((ns - 1) * source_distance / v_env) / dt
    )
elif alpha > 90:
    max_latency = np.cos(np.deg2rad(alpha)) * (source_distance / v_env) / dt

all_data = comm.gather(src.data, root=0)

if rank == 0:
    full_data = np.concatenate(all_data, axis=1)

    for i in range(full_data.shape[1]):
        latency = -np.cos(np.deg2rad(alpha)) * (i * source_distance / v_env)
        full_data[:, i] = np.roll(full_data[:, i], int(latency / dt + max_latency))
    
    divided_data = np.array_split(full_data, size, axis=1)
else:
    divided_data = None

new_data = comm.scatter(divided_data, root=0)
np.copyto(src.data, new_data)

[stderr:0] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


[stderr:1] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


In [10]:
%%px
sonar.u.data.fill(0)
sonar.op(time=sonar.time_range.num - 2, dt=dt)

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

[stderr:0] Operator `Kernel` ran in 7.57 s


[stderr:1] Operator `Kernel` ran in 7.57 s


Out[0:9]: 
PerformanceSummary([(PerfKey(name='section0', rank=0),
                     PerfEntry(time=2e-06, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section0', rank=1),
                     PerfEntry(time=2e-06, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section1', rank=0),
                     PerfEntry(time=7.465028999999961, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section1', rank=1),
                     PerfEntry(time=7.438919999999989, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section2', rank=0),
                     PerfEntry(time=0.02769299999999515, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section2', rank=1),
                     PerfEntry(time=0.04339200000000258, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[

Out[1:9]: 
PerformanceSummary([(PerfKey(name='section0', rank=0),
                     PerfEntry(time=2e-06, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section0', rank=1),
                     PerfEntry(time=2e-06, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section1', rank=0),
                     PerfEntry(time=7.465028999999961, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section1', rank=1),
                     PerfEntry(time=7.438919999999989, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section2', rank=0),
                     PerfEntry(time=0.02769299999999515, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[])),
                    (PerfKey(name='section2', rank=1),
                     PerfEntry(time=0.04339200000000258, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[

In [11]:
%%px
all_rec_data = comm.gather(sonar.rec.data, root=0)

# If current process is root
if rank == 0:
    # Concatenate the gathered data if needed
    full_rec_data = np.concatenate(all_rec_data, axis=1)

    # Perform a plot operation on the full_rec_data
    plt.plot(full_rec_data[:, 64])
    plt.show()

[stderr:0] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


[stderr:1] Pickling of `Data` objects is not supported. Casting to `numpy.ndarray`


%px:   0%|          | 0/2 [00:00<?, ?tasks/s]